# Practicing with Z3

> Before trying to solve the exercises contained in this notebook, the reader is suggested to have gained experience with the tool through the `Z3 Tutorial.ipynb` notebook.

In [1]:
from z3 import *

## N-Queens

Does not need presentation, right?

### Why not backtracking?

The following implementation is taken from [Rosetta Code](https://rosettacode.org/wiki/N-queens_problem#Python).

> Note: The `nqueens` function returns a generator and we only measure the time required to find the first feasible solution

In [2]:
def display_checkboard(s):
    board = [[0] * len(s) for i in range(len(s))]
    for x,y in s:
        board[x-1][y-1] = 1
    for i in range(len(board)):
        for j in range(len(board[0])):
            symbol = '♛' if board[i][j] == 1 else '.'
            print(symbol, end = ' ')
        print()

def under_attack(col, queens):
    return col in queens or \
           any(abs(col - x) == len(queens)-i for i, x in enumerate(queens))
 
def nqueens(n):
    solutions = [[]]
    for row in range(n):
        solutions = (solution+[i+1]
                       for solution in solutions # first for clause is evaluated immediately,
                                                 # so "solutions" is correctly captured
                       for i in range(n)
                       if not under_attack(i+1, solution))
    return solutions

In [3]:
%%time
generator = nqueens(26)
display_checkboard(list(enumerate(next(generator))))

. . ♛ . . . . . . . . . . . . . . . . . . . . . . . 
. . . . ♛ . . . . . . . . . . . . . . . . . . . . . 
. ♛ . . . . . . . . . . . . . . . . . . . . . . . . 
. . . ♛ . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . ♛ . . . . . . . . . . . . . . . . . 
. . . . . . . . . . ♛ . . . . . . . . . . . . . . . 
. . . . . . . . . . . . ♛ . . . . . . . . . . . . . 
. . . . . . . . . . . . . . ♛ . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . ♛ . . . . . 
. . . . . . . . . . . . . . . . . . . . . . ♛ . . . 
. . . . . . . . . . . . . . . . . . . . . . . . ♛ . 
. . . . . . . . . . . . . . . . . . . ♛ . . . . . . 
. . . . . . . . . . . . . . . . . . . . . ♛ . . . . 
. . . . . . . . . . . . . . . . . . . . . . . ♛ . . 
. . . . . . . . . . . . . . . . . . . . . . . . . ♛ 
. . . . . . . . . ♛ . . . . . . . . . . . . . . . . 
. . . . . . ♛ . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . ♛ . . . . . . . . . . 
. . . . . . . . . . . ♛ . . . . . . . . . . . 

We can notice that the solving time becomes quickly intractable as we increase $n$ using traditional backtracking. For instance, when $n > 30$ time exceeds the minutes barrier. 

This is due to the fact that the algorithm has a computational complexity of $O(n!)$. If you are interested in knowing why you can refer to this [article](http://techieme.in/solving-the-n-queen-problem/).

### SMT Solution

With Z3 you only take care of the constraints of your problem, leaving to the solver the burden of searching for a feasible solution.

In [4]:
def abs_z3(a):
    return If(a >= 0, a, -a)

def nqueens_z3(n):
    cols = [Int('c%d' % i) for i in range(n)]
    rows = range(n)
    s = Solver()

    for i in range(n):
        # each queen must be in the chessboard's limits
        s.add(cols[i] >= 0, cols[i] < n)

    for i in range(n - 1):
        for j in range(i + 1, n):
            s.add(cols[i] != cols[j])
            s.add(rows[i] != rows[j])
            s.add(abs_z3(cols[i] - cols[j]) != abs(rows[i] - rows[j]))

    if s.check() == unsat:
        raise Exception('Unsatisfiable constraints')

    m = s.model()
    return [(m[x].as_long(), y) for x, y in zip(cols, rows)]

In [5]:
%%time
display_checkboard(nqueens_z3(26))

. . . . . . . . . . . . ♛ . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . ♛ . . . . . 
. . . . . . . . . . . . . . . . . . . . . . ♛ . . . 
. . . . . . . . . . . ♛ . . . . . . . . . . . . . . 
. . . . . ♛ . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . ♛ . . . . . . . . . . . 
. . . . ♛ . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . ♛ 
. . . ♛ . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . ♛ . . . . . . . . . . . . 
. . . . . . . ♛ . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . ♛ . . . . . . . . 
. ♛ . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . ♛ . 
. . . . . . . . . . . . . . . . . . . . . ♛ . . . . 
. . . . . . . . . . . . . . . . . . ♛ . . . . . . . 
. . ♛ . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . ♛ . . . . . . 
. . . . . . . . . ♛ . . . . . . . . . . . . . 

Benefits of using Z3 are evident making a comparison with the previous execution time with the same instance size, and you can even try to increase $n$ to values up to 100 still obtaining a solution in less than a minute.

> Note: I suggest not to display the solution for $n>50$ because it would not fit the cell space well

## Sudoku

Another pretty famous game you are familiar with, right?

The goal is to insert the numbers in the boxes to satisfy the following conditions:

* Each row...
* Each column...
* Each 3x3 box...

... must contain the digits 1 through 9 exactly once.

This exercise demonstrates how to use list comprehensions, typical of Python, for encoding multiple constraints.

In [6]:
# sudoku instance, '0's correspond to empty cells
instance = ((0,0,0,0,9,4,0,3,0),
            (0,0,0,5,1,0,0,0,7),
            (0,8,9,0,0,0,0,4,0),
            (0,0,0,0,0,0,2,0,8),
            (0,6,0,2,0,1,0,5,0),
            (1,0,2,0,0,0,0,0,0),
            (0,7,0,0,0,0,5,2,0),
            (9,0,0,0,6,5,0,0,0),
            (0,4,0,9,7,0,0,0,0))

In [7]:
# 9x9 matrix of integer variables
X = [[Int("x_%s_%s" % (i+1, j+1)) for j in range(9)] for i in range(9)]

# each cell contains a value in {1, ..., 9}
cells_c = [And(1 <= X[i][j], X[i][j] <= 9)
           for i in range(9) for j in range(9)]

# each row contains a digit at most once
rows_c = [Distinct(X[i]) for i in range(9)]

# each column contains a digit at most once
cols_c = [Distinct([X[i][j] for i in range(9)]) for j in range(9)]

# each 3x3 square contains a digit at most once
sq_c = [Distinct([X[3*i0 + i][3*j0 + j] for i in range(3) for j in range(3)])
        for i0 in range(3) for j0 in range(3)]

sudoku_c = cells_c + rows_c + cols_c + sq_c

instance_c = [If(instance[i][j] == 0, True, X[i][j] == instance[i][j])
              for i in range(9) for j in range(9)]

s = Solver()
s.add(sudoku_c + instance_c)

if s.check() == sat:
    m = s.model()
    r = [ [ m.evaluate(X[i][j]) for j in range(9) ]
          for i in range(9) ]
    print_matrix(r)
else:
    print("Failed to solve")

[[7, 1, 5, 8, 9, 4, 6, 3, 2],
 [2, 3, 4, 5, 1, 6, 8, 9, 7],
 [6, 8, 9, 7, 2, 3, 1, 4, 5],
 [4, 9, 3, 6, 5, 7, 2, 1, 8],
 [8, 6, 7, 2, 3, 1, 9, 5, 4],
 [1, 5, 2, 4, 8, 9, 7, 6, 3],
 [3, 7, 6, 1, 4, 8, 5, 2, 9],
 [9, 2, 8, 3, 6, 5, 4, 7, 1],
 [5, 4, 1, 9, 7, 2, 3, 8, 6]]


## Einstein's Riddle

The author of this problem is Albert Einstein who said that 98% of the people in the world couldn't solve it.

### Facts:

1. There are 5 houses (along a street) in 5 different **colors**:
        Blue, Green, Red, White, and Yellow.

2. In each house lives a person of a different **nationality**:
        Brit, Dane, German, Norwegian, and Swede.

3. These 5 owners

    3.1. drink a certain **beverage**:
        Beer, Coffee, Milk, Tea, and Water.
    3.2. smoke a certain brand of **cigar**:
        Blue Master, Dunhill, Pall Mall, Prince, and blend,
    3.3. and keep a certain **pet**:
        Cat, Bird, Dog, Fish, and Horse.
4. **No owners have the same** pet, smoke the same brand of cigar, or drink the same beverage.

### Hints:

1.		The Brit lives in a red house.
2.		The Swede keeps dogs as pets.
3.		The Dane drinks tea.
4.		The green house is on the left of the white house (next to it).
5.		The green house owner drinks coffee.
6.		The person who smokes Pall Mall rears birds.
7.		The owner of the yellow house smokes Dunhill.
8.		The man living in the house right in the center drinks milk.
9.		The Norwegian lives in the first house.
10.		The man who smokes blend lives next to the one who keeps cats.
11.		The man who keeps horses lives next to the man who smokes Dunhill.
12.		The owner who smokes Blue Master drinks beer.
13.		The German smokes Prince.
14.		The Norwegian lives next to the blue house.
15.		The man who smokes blend has a neighbor who drinks water.

### Question

Who keeps the fish?

### Solution

The first step consists in encoding the variables of the problem inside Z3. We associate each of them to an integer between 0 and 4, and we also enforce that two houses do not contain the same property.

In [8]:
p = [('Yellow', 'Green', 'Blue', 'Red', 'White'),
    ('Brit', 'Dane', 'German', 'Norwegian', 'Swede'),
    ('Beer', 'Coffee', 'Milk', 'Tea', 'Water'),
    ('Blue Master', 'Dunhill', 'Pall Mall', 'Prince', 'Blend'),
    ('Cat', 'Bird', 'Dog', 'Fish', 'Horse')]

s = Solver()

houses = []
for n in range(5):
    house = []
    for prop in 'color', 'nationality', 'beverage', 'smoke', 'pet':
        x = Int(f"{prop}{n}")
        s.add(x >= 0, x <= 4)
        house.append(x)
    houses.append(house)

# Associate an alias number to each attribute for convenience
color, nationality, beverage, smoke, pet = range(5)

# Each column must have different values
for i in range(5):
    s.add(Distinct([houses[j][i] for j in range(5)]))

Now, let's feed the solver with the available hints as constraints:

> Remember that we can exploit the advantages provided us by Python to simplify this task

In [9]:
def same_house(prop1: int, value1: str, prop2: int, value2: str):
    return Or([
        And(houses[i][prop1] == p[prop1].index(value1), houses[i][prop2] == p[prop2].index(value2))
        for i in range(5)
    ])

def next_to(prop: int, value1: str, value2: str):
    return Or([
        And(houses[i][prop] == p[prop].index(value1), houses[i + 1][prop] == p[prop].index(value2))
        for i in range(4)
    ])

def neighbor(prop1: int, value1: str, prop2: int, value2: str):
    return Or(
        And(houses[0][prop1] == p[prop1].index(value1), houses[1][prop2] == p[prop2].index(value2)),
        And(houses[1][prop1] == p[prop1].index(value1), Or(houses[0][prop2] == p[prop2].index(value2), houses[2][prop2] == p[prop2].index(value2))),
        And(houses[2][prop1] == p[prop1].index(value1), Or(houses[1][prop2] == p[prop2].index(value2), houses[3][prop2] == p[prop2].index(value2))),
        And(houses[3][prop1] == p[prop1].index(value1), Or(houses[2][prop2] == p[prop2].index(value2), houses[4][prop2] == p[prop2].index(value2))),
        And(houses[4][prop1] == p[prop1].index(value1), houses[3][prop2] == p[prop2].index(value2)),
    )

In [10]:
# 1. The Brit lives in a red house.
s.add(same_house(nationality, 'Brit', color, 'Red'))

# 2. The Swede keeps dogs as pets.
s.add(same_house(nationality, 'Swede', pet, 'Dog'))

# 3. The Dane drinks tea.
s.add(same_house(nationality, 'Dane', beverage, 'Tea'))

# 4. The green house is on the left of the white house (next to it).
s.add(next_to(color, 'Green', 'White'))

# 5. The green house owner drinks coffee.
s.add(same_house(color, 'Green', beverage, 'Coffee'))

# 6. The person who smokes Pall Mall rears birds.
s.add(same_house(smoke, 'Pall Mall', pet, 'Bird'))

# 7. The owner of the yellow house smokes Dunhill.
s.add(same_house(color, 'Yellow', smoke, 'Dunhill'))

# 8. The man living in the house right in the center drinks milk.
s.add(houses[2][beverage] == p[beverage].index('Milk'))

# 9. The Norwegian lives in the first house.
s.add(houses[0][nationality] == p[nationality].index('Norwegian'))

# 10. The man who smokes blend lives next to the one who keeps cats.
s.add(neighbor(smoke, 'Blend', pet, 'Cat'))

# 11. The man who keeps horses lives next to the man who smokes Dunhill.
s.add(neighbor(pet, 'Horse', smoke, 'Dunhill'))

# 12. The owner who smokes Blue Master drinks beer.
s.add(same_house(smoke, 'Blue Master', beverage, 'Beer'))

# 13. The German smokes Prince.
s.add(same_house(nationality, 'German', smoke, 'Prince'))

# 14. The Norwegian lives next to the blue house.
s.add(neighbor(nationality, 'Norwegian', color, 'Blue'))

# 15. The man who smokes blend has a neighbor who drinks water.
s.add(neighbor(smoke, 'Blend', beverage, 'Water'))

s.check()

sat

If the hints have been encoded without errors, you should have obtained a `sat` result from the previous cell.

We just have to show the corresponding solution and look for the owner of the house containing the fish.

In [11]:
m = s.model()
solution = [[m[case].as_long() for case in line] for line in houses]

for i in range(5):
    print('Color: %s, Nationality: %s, Beverage: %s, Smoke: %s, Pet: %s' % (
        p[color][solution[i][color]],
        p[nationality][solution[i][nationality]],
        p[beverage][solution[i][beverage]],
        p[smoke][solution[i][smoke]],
        p[pet][solution[i][pet]])
    )

Color: Yellow, Nationality: Norwegian, Beverage: Water, Smoke: Dunhill, Pet: Cat
Color: Blue, Nationality: Dane, Beverage: Tea, Smoke: Blend, Pet: Horse
Color: Red, Nationality: Brit, Beverage: Milk, Smoke: Pall Mall, Pet: Bird
Color: Green, Nationality: German, Beverage: Coffee, Smoke: Prince, Pet: Fish
Color: White, Nationality: Swede, Beverage: Beer, Smoke: Blue Master, Pet: Dog


## Magic square

In [12]:
def magic_square(size):
    """Try to find a solution for a size-magic square"""
    def column(matrix, i):
        """Get the column i of matrix"""
        return [matrix[j][i] for j in range(size)]

    def get_diagonals(matrix):
        """Get the diagonals of matrix"""
        return [matrix[i][i] for i in range(size)], [matrix[i][size - i - 1] for i in range(size)]

    def get_constrained_int(x, y, s):
        """Get an Int and add the constraints associated directly in the solver"""
        # Int() is too slow!
        x = BitVec('x%dy%d' % (x, y), 32)
        s.add(x > 0, x <= size**2)
        return x

    s = Solver()
    magic = (size * (size**2 + 1)) / 2
    matrix = [[get_constrained_int(y, x, s) for y in range(size)] for x in range(size)]

    # Each value must be different
    s.add(Distinct([matrix[i][j] for j in range(size) for i in range(size)]))

    for i in range(size):
        # Sum of each line, column must be equal to magic
        s.add(Sum(matrix[i]) == magic, Sum(column(matrix, i)) == magic)

    # Sum of each diagonal must be equal to magic
    d1, d2 = get_diagonals(matrix)
    s.add(Sum(d1) == magic, Sum(d2) == magic)

    if s.check() == unsat:
        raise Exception('The problem is not sat')

    m = s.model()
    return [[m[val].as_long() for val in line] for line in matrix], magic

def display_magic_square(s, magic):
    """Display the magic square with the solution"""
    print(f'Magic value: {magic}')
    for i in range(len(s)):
        print(('%.3d|' * len(s)) % tuple(s[i]))

s, magic = magic_square(3)
display_magic_square(s, magic)

Magic value: 15.0
006|001|008|
007|005|003|
002|009|004|
